### Chapter 5 Arrays 

Oftentimes, arrays problems require some level of manipulation in place
that involves things like updating, deleting, swapping, etc...

Hints about array libraries 
- syntax of list types `[1, 2, 3,4]`
- operations .append(), .remove(), .insert(pos, item)
- 2D `[[][][]]
- Value check O(n)
- deep vs shallow B=A , B = list(A), .copy, .deepcopy()
- key methods min,max, bisect, reverse vs reversed
- slicing `A[2:4]`
- list comprehensiosn 
    - input sequence
    - iterator
    - logical condition
    - yielded expression
   


#### 5.1 The Dutch National Flag Problem 
The Dutch national flag partitioning is when you have a pivot in which there are three subarrays. Those that are smaller than the pivot, equal to the pivot
and greater than the pivot. This helps with the distribution of subarrays given a pivot 

Write a program that takes an array A and an index i into A, and rearrange the elements such that all elements follow the Dutch national flg partitioning. 

In [34]:
#attempt
def partition(A, i):
  def swap(A, i, j):
    temp = A[i]
    A[i] = A[j]
    A[j] = temp
  pivot = A[i]
  A[i] = A[-1] # swap the pivot with the last element
  i = 0
  j = len(A) - 2
  while i <= j: 
    if A[i] < pivot:
      i += 1
    else:
      swap(A, i, j)
      j -= 1

  j = len(A) - 2
  while i <= j:
    if A[i] == pivot:
      i += 1
    else:
      swap(A, i, j)
      j -= 1
  A[-1] = pivot
  swap(A, len(A) - 1, i)
# tests
ex1 = [0, 1, 2, 0, 2, 1, 1]
partition(ex1, 3)
print(ex1)
partition(ex1, 2)
print(ex1)

# The runtime for this code is `O(n)`

0
[0, 0, 2, 1, 2, 1, 1]
2
[0, 0, 1, 1, 1, 2, 2]


##### Solutions 
---
##### General Approach 
The first general approach to this problem is to have two passes where we add elements that are less than pivot to the front
then those greater to the back

```python
RED, WHITE, BLUE = range(3)

def dutch_flag_partition(pivot_index, A):
    pviot = A[pivot_index]
    # First pass: group elements smaller than pivot. 
    for i in range(len(A)):
        #Look for a smaller element
        for j in range(i + 1, len(A)):
            if A[j] < pivot: 
                A[i], A[j] = A[j], A[i]
                break
    #Second Pass: group elements larger than pivot
    for i in reversed(range(len(A))):
        if A[i] < pivot:
            break
        # Look for a larger element. Stop when we reach an element less than
        # pivot, since first pass has moved them to the start of A. 
        for j in reversed(range(i)):
            if A[j] > pivot:
                A[i], A[j] = A[j], A[i]
                break
```

The runtime for the following equation is `O(n^2)` with space complexity of `O(1)`

The second example adopts a similar approach of moving elements in two pass but simply moves out-of-place elements 
as soon as it's found. 

```python
RED, WHITE, BLUE = range(3)

def dutch_flag_partition(pivot_index, A):
    pviot = A[pivot_index]
    # First pass: group elements smaller than pivot. 
    smaller = 0
    for i in range(len(A)):
        if A[i] < pivot:
            A[i], A[smaller] = A[smaller], A[i]
            smaller += 1
    # Second pass: group elements larger than pivot. 
    larger = len(A) - 1
    for i in range(len(A)):
        if A[i] < pivot:
            break
        elif A[i] > pivot:
            A[i], A[larger] = A[larger], A[i]
            larger -= 1
```
The runtime for the following equation is `O(n)` with space complexity of `O(1)`

The final approach is uses a single passes that keeps track of four subarrays, <, ==, >, and unclassifed

```python
RED, WHITE, BLUE = range(3)

def dutch_flag_partition(pivot_index, A):
    pviot = A[pivot_index]
    # Keep the following invariants during partitioning:
    # botton group: A[:smaller].
    # middle group: A[smaller:equal]
    # unclassifed group: A[equal:larger]
    # top group: A[larger:]
    smaller, equal, larger = 0, 0, len(A)
    # keep iterating as long as there is an unclassified element
    while equal < larger:
        #A[equal] is the incoming unclassifed element
        if A[equal] < pivot:
            A[smaller], A[equal] = A[equal], A[smaller]
            smaller, equal = smaller + 1, equal + 1
        elif A[equal] == pivot:
            equal += 1
        else: #A[equal] > pivot
            larger -= 1
            A[equal], A[larger] = A[larger], A[equal]
```

The following algorithm has time complexity of `O(n)` and space complexity of `O(1)`

**Note: The following switching syntax is useful `A[smaller], A[equal] = A[equal], A[smaller]`**

##### Variant: 
Assuming that keys take one of three values, reoder the array so that all objects with the same key appear together.

In [36]:
RED, WHITE, BLUE  = range(3)

def dutch_flag_partition(A):
  red_whiteB, white_blueB, end = 0, 0, len(A)
  while(white_blueB != end):
    # checks if unclassified is red
    # red = A[:red_whiteB]
    # white = A[red_whiteB:white_blueB]
    # unclassied = A[white_blueB:end]
    # blue = [end:]
    if (A[white_blueB] == RED):
      A[white_blueB], A[red_whiteB] = A[red_whiteB], A[white_blueB]
      white_blueB, red_whiteB = white_blueB + 1, red_whiteB + 1
    elif(A[white_blueB] == WHITE):
      white_blueB += 1
    else:
      end -= 1
      A[end], A[white_blueB] = A[white_blueB], A[end]

# tests 
A = [0, 1, 2, 0, 0, 2, 1, 2, 0]
dutch_flag_partition(A)
print(A)

[0, 0, 0, 0, 1, 1, 2, 2, 2]


##### Variant: 
Given an array A of n objects with keys that takes one of four vlaues, reorder the array that all objects that have the same key appear together. Use O(1) additonal space and O(n) time.

In [44]:
RED, WHITE, BLUE, BLACK  = range(4)

def dutch_flag_partition(A):
  red_whiteB, white_blueB, blue_blackB, end = 0, 0, len(A), len(A)
  while(white_blueB != blue_blackB):
    # checks if unclassified is red
    # red = A[:red_whiteB]
    # white = A[red_whiteB:white_blueB]
    # unclassied = A[white_blueB:blue_blackB]
    # blue = [blue_blackB:end]
    # black = [end:]
    if (A[white_blueB] == RED):
      A[white_blueB], A[red_whiteB] = A[red_whiteB], A[white_blueB]
      white_blueB, red_whiteB = white_blueB + 1, red_whiteB + 1
    elif(A[white_blueB] == WHITE):
      white_blueB += 1
    elif(A[white_blueB] == BLUE):
      blue_blackB -= 1
      A[white_blueB], A[blue_blackB] = A[blue_blackB], A[white_blueB]
    else: 
      blue_blackB -= 1
      A[white_blueB], A[blue_blackB] = A[blue_blackB], A[white_blueB]
      end -= 1
      A[blue_blackB], A[end] = A[end], A[blue_blackB]

# tests 
A = [0, 3, 2, 0, 0, 2, 1, 2, 3]
dutch_flag_partition(A)
print(A)

[0, 0, 0, 1, 2, 2, 2, 3, 3]


##### Variant: 
Given em array A of n objects with Boolean-valued keys, reorder the array so that objects
that have the key false appear first. The relative ordering of objects *with key true* should not change.
Use O(1) additional space and O(n) time.

In [48]:
def dutch_flag_partition(A):
  start, end = len(A) - 1, len(A) - 1
  while (start >= 0):
    if (A[start]):
      A[start], A[end] = A[end], A[start]
      end -= 1
    start -= 1
#tests
A = [True, False, False, True, True, False, True, False, True, True]
dutch_flag_partition(A)
print(A)

[False, False, False, False, True, True, True, True, True, True]


#### 5.2 The Increment An Arbitrary-Precision Integer 
Write a program which takes as input an array of digits encoding a nonnegative decimal integer D and updates teh array to represent the integer D + 1. 
For example <1, 2, 9> should update to <1, 3, 0>

In [52]:
# Attempt 
def bigIntAdd1(A):
  carry = 0
  A[-1] += 1
  for i in reversed(range(len(A))):
    total = A[i] + carry
    A[i], carry = total % 10, total // 10
    if not(carry):
      break
  if (carry):
    A.insert(0, carry)
# tests 
A = [1, 2, 9]
bigIntAdd1(A)
print(A)

# The time complexity is O(n)

[1, 3, 0]


##### Solutions 
---
##### In place modification
The solutions use a similar approach to my attempt except it focuses on +1 and modifies two elements each iteration

```python
def plus_one(A):
  A[-1] += 1
  for i in reversed(range(1, len(A))):
    if A[i] != 10:
      break
    A[i] = 0
    A[i - 1] += 1
  if A[0] == 10:
    # There is a carry-out, so we need one more digit to store teh result.
    # A slick way to do this is to append a 0 at the end of the array, 
    # and update teh first entry to 1. 
    A[0] = 1
    A.append(0)
  return A
```

The time complexity is `O(n)`, where n is the length of A. 


##### Variant: 
Write a program which takes as input two strings `s` and `t` of bits encoding binary numbers `B_s` and `B_t`, respectively, and returns a new string of bits
representing the number `B_s + B_t`. 

In [55]:
def addStrBits(s, t):
  newStr = ""
  sL, tL = len(s), len(t)
  carryout = 0
  while sL and tL != 0:
    sL, tL = sL - 1, tL-1
    newStr = str(carryout ^ int(s[sL]) ^ int(t[tL])) + newStr
    carryout = (int(s[sL]) & int(t[tL])) | (int(s[sL]) & carryout) | (int(t[tL]) & carryout)

  while sL != 0:
    sL -= 1
    newStr = str(carryout ^ int(s[sL]))  + newStr
    carryout = (int(s[sL]) & carryout)

  while tL != 0:
    tL -= 1
    newStr = str(carryout ^ int(t[tL])) + newStr
    carryout = (int(t[tL]) & carryout)

  if carryout:
    newStr = '1' + newStr
  return newStr

# test
print(addStrBits('01101101', '10111'))   

TypeError: unsupported operand type(s) for ^: 'int' and 'str'